In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
import pickle
from tqdm import tqdm

import sys
sys.path.insert(0, '../')
from utils.evaluation import read_base_results, read_drp_results, read_eval_results

sns.set()
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

%load_ext autoreload
%autoreload 2

In [ ]:
def find_rashomon_model_ratio(eps_list, base_result, sampling_result, bdrp_result, gdrp_result):
    sampling_loss = sampling_result['all_sampling_test_loss']
    nsampling = sampling_result['all_sampling_test_loss'].shape[0]
    
    for i, eps in enumerate(eps_list):
        nsampling_r = np.sum(sampling_loss<=base_result['test_loss']+eps) / nsampling

        eps_mean = (bdrp_result['drp_test_loss']-base_result['test_loss']).mean(axis=1)
        idx = np.arange(len(eps_mean))[eps_mean <= eps][-1]
        n_bdrp = (idx+1)*bdrp_result['drp_test_loss'].shape[1]
        n_bdrp_r = np.sum(bdrp_result['drp_test_loss'][:idx, :] <= base_result['test_loss']+eps) / n_bdrp
        
        eps_mean = (gdrp_result['drp_test_loss']-base_result['test_loss']).mean(axis=1)
        idx = np.arange(len(eps_mean))[eps_mean <= eps][-1]
        n_gdrp = (idx+1)*gdrp_result['drp_test_loss'].shape[1]
        n_gdrp_r = np.sum(gdrp_result['drp_test_loss'][:idx, :] <= base_result['test_loss']+eps) / n_gdrp

        print('& ${:.3f}$ & ${:2.2f}$\% & ${:2.2f}$\% & ${:2.2f}$\% \\\\'.format(eps, nsampling_r*100, n_bdrp_r*100, n_gdrp_r*100))

    return 

In [ ]:
eps_list = np.linspace(0.001, 0.004, 4)

## Credit Approval Dataset

In [ ]:
datasetname='credit-approval'
file_path = '../results/'

In [ ]:
base_file = datasetname+'-base-100-1-1000'
sampling_filename = datasetname+'-sampling-100-20,25,30,35,40,45,50,55-eval'
bdrp_train_file = datasetname+'-bernoulli-dropout-100-1-1000-100-0.2'
gdrp_train_file = datasetname+'-gaussian-dropout-100-1-1000-100-0.6'

In [ ]:
base_result = read_base_results(file_path+base_file)
sampling_result = np.load(file_path+sampling_filename+'.npz')
bdrp_result = read_drp_results(file_path+bdrp_train_file)
gdrp_result = read_drp_results(file_path+gdrp_train_file)

In [ ]:
find_rashomon_model_ratio(eps_list, base_result, sampling_result, bdrp_result, gdrp_result)